Welcome! The purpose of this project is to apply markov chains to the S&P 500 to try and predict whether it will be increasing, decreasing, or remain stable on any given day based on historical data.

The purpose of the following block is to dowload the historical data of the S&P 500 and save it to a csv file. 

In [2]:
import yfinance as yf
import pandas as pd
import os 

ticker = "^GSPC"
end_date = "2025-12-01"

file_path = os.path.expanduser("~/s&p_500_data.csv")

sp500_data = yf.download(ticker, period="max", end=end_date, progress=False)

sp500_data.to_csv(file_path, index=True)

verified_data = pd.read_csv(
    file_path, 
    index_col=0, 
    parse_dates=True, 
    date_format='%Y-%m-%d' 
)

verified_data['Close'] = pd.to_numeric(verified_data['Close'], errors='coerce')

print(verified_data.head())



/var/folders/06/fqzvbskd6qgg7v6j_hn8x_8r0000gn/T/ipykernel_13711/1743721159.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500_data = yf.download(ticker, period="max", end=end_date, progress=False)


                Close                High                 Low  \
Price                                                           
Ticker            NaN               ^GSPC               ^GSPC   
Date              NaN                 NaN                 NaN   
1927-12-30  17.660000   17.65999984741211   17.65999984741211   
1928-01-03  17.760000  17.760000228881836  17.760000228881836   
1928-01-04  17.719999  17.719999313354492  17.719999313354492   

                          Open Volume  
Price                                  
Ticker                   ^GSPC  ^GSPC  
Date                       NaN    NaN  
1927-12-30   17.65999984741211      0  
1928-01-03  17.760000228881836      0  
1928-01-04  17.719999313354492      0  


This block creates a new column with the Daily Return (%) of the S&P 500 and displays it.

In [3]:

verified_data['Daily Return (%)'] = verified_data['Close'].pct_change() * 100

print(verified_data['Daily Return (%)'])

Price
Ticker             NaN
Date               NaN
1927-12-30         NaN
1928-01-03    0.566254
1928-01-04   -0.225230
                ...   
2025-11-21    0.982304
2025-11-24    1.546722
2025-11-25    0.906170
2025-11-26    0.690671
2025-11-28    0.535477
Name: Daily Return (%), Length: 24597, dtype: float64


Here, I am calculating the average daily return and the standard deviation using the daily return values we found in the previous block.

In [4]:
average_daily_return = verified_data['Daily Return (%)'].mean()

print(f"Average Daily Return: {average_daily_return:.5}%")

standard_deviation_return = verified_data['Daily Return (%)'].std()

print(f"\nStandard Deviation: {standard_deviation_return:.5f}%")

Average Daily Return: 0.031375%

Standard Deviation: 1.19351%


Here, I am setting the thresholds for what will determine if a state is considered increasing, decreasing, or stable. I used a threshold of +/- 0.1194% (0.1 standard deviations) for the 'Stable' state. With increases or decreases beyond the threshold being classified as their respective states. Using 0.1 standard deviations is large enough that most  days are not automatically classified as increasing or decreasing but small enough that not every day is automatically classisfied as stable either. 

In [5]:
standard_deviation_return = verified_data['Daily Return (%)'].std()

# Define the threshold based on 10% of the Standard Deviation
THRESHOLD = 0.1 * standard_deviation_return 

print(f"Using a threshold of +/- {THRESHOLD:.4f}% ({0.1 * standard_deviation_return.round(4):.1f} standard deviations) for the 'Stable' state.")

# Initialize the 'Market State' column by classifying everything as 'Same (S)' (the default neutral state)
verified_data['Market State'] = 'Stable (S)'

# Classify 'Increase'
verified_data.loc[
    verified_data['Daily Return (%)'] > THRESHOLD, 
    'Market State'
] = 'Increase (I)'

# Classify 'Decrease'
verified_data.loc[
    verified_data['Daily Return (%)'] < -THRESHOLD, 
    'Market State'
] = 'Decrease (D)'

print(verified_data[['Daily Return (%)', 'Market State']].head())

Using a threshold of +/- 0.1194% (0.1 standard deviations) for the 'Stable' state.
            Daily Return (%)  Market State
Price                                     
Ticker                   NaN    Stable (S)
Date                     NaN    Stable (S)
1927-12-30               NaN    Stable (S)
1928-01-03          0.566254  Increase (I)
1928-01-04         -0.225230  Decrease (D)


Here, I am creating the transition matrix based on the historical market states. 

In [6]:
# Shift the 'Market State' column up by one row
verified_data['Next State'] = verified_data['Market State'].shift(-1)

# Drop the last row (which has NaN for 'Next State') and the first row (NaN return)
transition_df = verified_data.dropna(subset=['Next State', 'Market State'])

# Calculate the Transition Probability Matrix (TPM) using crosstab
# 'normalize="index"' divides each count by the row total (Current State total)
tpm = pd.crosstab(
    transition_df['Market State'], 
    transition_df['Next State'], 
    normalize='index' # Normalize by row
)

# Print the result
print("\n--- Transition Probability Matrix  ---")
print("Rows = Current State | Columns = Next Day's State")
print(tpm.round(5))


--- Transition Probability Matrix  ---
Rows = Current State | Columns = Next Day's State
Next State    Decrease (D)  Increase (I)  Stable (S)
Market State                                        
Decrease (D)       0.43392       0.44704     0.11904
Increase (I)       0.37268       0.47603     0.15129
Stable (S)         0.39007       0.43599     0.17394


Here, we use the formula P^(n) = P^(0) * T^n to determine the probability matrix on any given (n) day based on the initial state P^(0) (stable) and the transition matrix of the n-day which is calcuated by taking the n-power of the transition probability matrix.

In [7]:
import numpy as np

def forecast_n_days(n_days: int) -> pd.Series:
    """
    Calculates the probability distribution of the market state 
    after n_days (P^(n) = P^(0) * T^n), starting from the first 
    historical state in the transition data.
    """
        
    # Identify the Initial State (P^(0))
    first_state = transition_df['Market State'].iloc[0]
    state_labels = tpm.index.tolist()
    
    # Construct Initial Probability Vector
    initial_state_vector = np.zeros(len(state_labels))
    first_state_index = state_labels.index(first_state) 
    initial_state_vector[first_state_index] = 1.0

    # Calculate n-Step Transition Matrix (T^n)
    T = tpm.values
    
    # Function call 
    T_n = np.linalg.matrix_power(T, n_days) 

    # Calculate the final probability distribution P^(n) 
    prob_n_days = np.dot(initial_state_vector, T_n)

    # Format Results 
    results = pd.Series(
        prob_n_days,
        index=tpm.columns,
        name=f'Forecast Probability after {n_days} days'
    )
    
    return results.round(4)

This would be the probability forecast for day one which is equivalent to the third row of the transition probability matrix since it would be P^(0) * T^(1).

In [8]:
print(forecast_n_days(1))

Next State
Decrease (D)    0.3901
Increase (I)    0.4360
Stable (S)      0.1739
Name: Forecast Probability after 1 days, dtype: float64


For day 2.

In [9]:
print(forecast_n_days(2))

Next State
Decrease (D)    0.3996
Increase (I)    0.4578
Stable (S)      0.1426
Name: Forecast Probability after 2 days, dtype: float64


For day 4.

In [10]:
print(forecast_n_days(4))

Next State
Decrease (D)    0.3996
Increase (I)    0.4588
Stable (S)      0.1416
Name: Forecast Probability after 4 days, dtype: float64


For day 100. By this time we can see that we have reached the steady state distributuon which is the probability that the market will be in each state on any given day.

In [11]:
print(forecast_n_days(100))

Next State
Decrease (D)    0.3996
Increase (I)    0.4588
Stable (S)      0.1416
Name: Forecast Probability after 100 days, dtype: float64
